# Методы. Часть 2

## Статический метод класса @staticmethod

In [1]:
class Human():

    def __init__(self, name, age=0):
        self.name = name
        self.age = age

    # Декоратор @staticmethod является встроенным в Python, его не нужно ни откуда импортировать.
    # Статический метод принимает только те аргументы, которые ему передают,
    # ссылки на объект (self) не существует внутри такого метода. Этот декоратор просто вырезает ссылку на объект.
    # Эту функцию, мы могли бы определить вне контекста класса и не использовать декоратор, но допустим,
    # мы посчитали, что так будет удобней.
    @staticmethod
    def is_age_valid(age):
        return 0 < age < 150 # age between 0 and 150

    def is_age_valid2(age):
        return 0 < age < 150 # age between 0 and 150

К статическому методу можно обращаться относительно класса или экземпляра класса:

In [2]:
Human.is_age_valid(35)

True

In [3]:
human = Human('Old Bobby')
human.is_age_valid(234)

False

Однако, без декоратора `@staticmethod` к методу можно обратится только через класс, а через экземпляр класса вызовет ошибку `TypeError`, так как метод экземпляра класса обязан принимать в качестве первого аргумента ссылку на экземпляр класса (self):

In [4]:
Human.is_age_valid2(35)

True

In [5]:
human.is_age_valid2(234)

TypeError: is_age_valid2() takes 1 positional argument but 2 were given

## Вычисляемые свойства класса (класс property)

`Property` позволяют изменять поведение и выполнять какую-то вычислительную работу при обращении, изменении, удалении атрибута экземпляра класса.

Рассмотрим пример, когда класс `property` не используется и к каким проблемам это приводит:

In [11]:
class Robot():

    def __init__(self, power):
        self.power = power


wall_e = Robot(100) # Инициализируем значением атрибут экземпляра класса
wall_e.power = 200  # Изменяем значение атрибута экземпляра класса

print(wall_e.power)

200


Допустим, вы заметили, что другие программисты ставят вашему роботу отрицательную мощность:

In [12]:
wall_e.power = -20

print(wall_e.power)

-20


Это не валидное значение для мощности робота и вам бы хотелось, чтобы когда устанавливалось новое значение
с отрицательным значением, то оно приводилось бы к нулю. Как можно поступить в таком случае? Например, мы можем добавить метод `set_power` для валидации значения:

In [13]:
class Robot():

    def __init__(self, power):
        self.power = power

    def set_power(self, power):
        if power < 0:
            self.power = 0
        else:
            self.power = power


wall_e = Robot(100)
wall_e.set_power(-20)

print(wall_e.power)

0


Однако, здесь есть небольшой нюанс. В таком случае, программисту нужно будет менять свой код, т.е., это потребует не только рефакторинга кода от вас, но и от других программистов использующих этот класс.  
Есть ли способ проще, еффективней? Есть, это использовать `property`:

In [14]:
class Robot():

    def __init__(self, power):
        self._power = power # Это просто какой-то тип, например число

    # Атрибут power теперь является объектом встроенного класса property(),
    # который является реализацией концепта Descriptor.
    # У этого property есть 3 метода: getter, setter и deleter.
    # Атрибут power без нижнего подчеркивания доступен из вне, это что-то вроде
    # прокси атрибута к "скрытому" атрибуту self._power:
    power = property()

    @power.getter
    def power(self):
        return self._power

    @power.setter
    def power(self, value):
        if value < 0:
            self._power = 0
        else:
            self._power = value

    @power.deleter
    def power(self):
        print('make robot useless')
        del self._power


wall_e = Robot(100)
wall_e.power = -20  # Сейчас, отрицательное значение не установить атрибуту power

print(wall_e.power)

0


Однако, приватный атрибут нам попрежнему доступен и в него мы можем установить отрицательное значение:

In [15]:
wall_e._power = -20 # Устанавливаем значение напрямую "приватному" атрибуту

print(wall_e.power)

-20


Если мы попробуем удалить атрибут `power`, то получим сообщение `make robot useless`, а затем только он будет удален командой `del`. Если мы не хотим его не удалять, то достаточно не вызывать оператор `del` в методе `@power.deleter`:

In [16]:
del wall_e.power

make robot useless


## Декоратор @property

Иногда, нужно как-то модифицировать чтение атрибута и выполнять какую-то полезную работу при чтении и это единственное что вам требуется, т.е., не нужно менять поведение при изменении, удалении значения атрибута. В таком случае есть более короткая запись (декоратор @property):

In [17]:
class Robot():

    def __init__(self, power):
        self._power = power

    # @property - встроенный декоратор. Реализует геттер атрибута объекта, аналог @power.getter
    @property
    def power(self):
        return self._power + 10


wall_e = Robot(200)

print(wall_e.power)

210


## Итоги

* Узнали, что такое статический метод (@staticmethod)
* Узнали, что такое свойство класса (@property)